In [1]:
# Imports
import pandas as pd
from tqdm import tqdm

In [96]:
# Functions
def translate(seq, translate_dict):  #translates biological sequences given a dictionary
    seq = seq.upper()
    codon_seq = ''
    for i in range(0, len(seq), 3):
        codon = seq[i:i + 1]
        codon_seq += translate_dict[codon]
    return codon_seq[:-1]

def extract_id(header):  # for getting uniprot sequences
    return header.split('|')[1]

def mismatch_count(seqa, seqb):
    mismatches = []
    for i, (aminoa, aminob) in enumerate(zip(seqa, seqb)):
        if aminoa != aminob:
            mismatches.append([aminoa, aminob, i])
    return mismatches

In [7]:
# Load mapping data
names = ['UniProtKB-AC', 'UniProtKB-ID', 'GeneID (EntrezGene)', 'RefSeq', 'GI', 'PDB', 'GO', 'UniRef100',
         'UniRef90', 'UniRef50', 'UniParc', 'PIR', 'NCBI-taxon', 'MIM', 'UniGene', 'PubMed', 'EMBL',
         'EMBL-CDS', 'Ensembl', 'Ensembl_TRS', 'Ensembl_PRO', 'Additional_PubMed']
df = pd.read_csv('/lustre/gleghorn/users/3018/idmapping_selected.tab', delimiter='\t', names=names, dtype=str)

In [13]:
df.head()

,UniProtKB-AC,UniProtKB-ID,GeneID (EntrezGene),RefSeq,GI,PDB,GO,UniRef100,UniRef90,UniRef50,...,NCBI-taxon,MIM,UniGene,PubMed,EMBL,EMBL-CDS,Ensembl,Ensembl_TRS,Ensembl_PRO,Additional_PubMed
0,Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,NaN,GO:0046782,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4,...,654924,NaN,NaN,15165820,AY548484,AAT09660.1,NaN,NaN,NaN,NaN
1,Q6GZX3,002L_FRG3G,2947774,YP_031580.1,49237299; 81941548,NaN,GO:0033644; GO:0016020,UniRef100_Q6GZX3,UniRef90_Q6GZX3,UniRef50_Q6GZX3,...,654924,NaN,NaN,15165820,AY548484,AAT09661.1,NaN,NaN,NaN,NaN
2,Q197F8,002R_IIV3,4156251,YP_654574.1,109287880; 123808694; 106073503,NaN,NaN,UniRef100_Q197F8,UniRef90_Q197F8,UniRef50_Q197F8,...,345201,NaN,NaN,16912294,DQ643392,ABF82032.1,NaN,NaN,NaN,NaN
3,Q197F7,003L_IIV3,4156252,YP_654575.1,106073504; 109287881; 123808693,NaN,NaN,UniRef100_Q197F7,UniRef90_Q197F7,UniRef50_Q197F7,...,345201,NaN,NaN,16912294,DQ643392,ABF82033.1,NaN,NaN,NaN,NaN
4,Q6GZX2,003R_FRG3G,2947775,YP_031581.1,81941547; 49237300,NaN,NaN,UniRef100_Q6GZX2,UniRef90_Q6GZX2,UniRef50_Q6GZX2,...,654924,NaN,NaN,15165820,AY548484,AAT09662.1,NaN,NaN,NaN,NaN


In [14]:
# Only get mappings between Uniprot and Ensembl transcripts, remove NANs
print(len(df)) 
df2 = df[['UniProtKB-AC', 'Ensembl_TRS']]
df2.dropna(how='any', inplace=True)
print(len(df2))

249877975
8887128


/opt/shared/anaconda/5.3.1-python3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df2.head()

,UniProtKB-AC,Ensembl_TRS
335,Q5ZLQ6,ENSGALT00010053687; ENSGALT00015040050
337,P31946,ENST00000353703.9; ENST00000372839.7
338,Q4R572,ENSMFAT00000073733
339,Q9CQV8,ENSMUST00000018470.10
340,A4K2U9,ENSPPYT00000012812.3


In [21]:
# Write to more manegable file
with open('/lustre/gleghorn/users/3018/ensembl_to_uniprot.csv', 'w') as f:
    for i in tqdm(range(len(df2))):
        if ';' in str(df2['Ensembl_TRS'].iloc[i]):
            f.write(str(df2['UniProtKB-AC'].iloc[i]) + ',' + str(df2['Ensembl_TRS'].iloc[i]).split(';')[0] + '\n')
        else:
            f.write(str(df2['UniProtKB-AC'].iloc[i]) + ',' + str(df2['Ensembl_TRS'].iloc[i]) + '\n')

100%|██████████| 8887128/8887128 [04:11<00:00, 35392.93it/s]


In [33]:
# Make preprocessed into fasta
with open('/lustre/gleghorn/users/3018/preprocessed_cds.txt', 'r') as a, open('/lustre/gleghorn/users/3018/preprocessed_cds.fasta', 'w') as b:
    lines = a.readlines()
    for i in tqdm(range(len(lines))):
        if (i+1) % 2 == 0:
            b.write(lines[i])
        else:
            b.write('>' + lines[i])

100%|██████████| 18881356/18881356 [00:14<00:00, 1296423.27it/s]


In [50]:
class sequence_dict(dict):
    def __init__(self):
        self = dict()
 
  # Function to add key:value
    def add(self, key, value):
        self[key] = value

seq_dict = sequence_dict()

# Make a dictionary of all Ensembl codon sequences
with open('/lustre/gleghorn/users/3018/preprocessed_cds.fasta', 'r') as f:
    lines = f.readlines()
    for i in tqdm(range(len(lines))):
        if (i+1) % 2 == 0:
            seq = lines[i].replace('\n', '').replace('>', '')
            seq_dict.add(key, seq)
        else:
            key = lines[i].replace('\n', '')
        

100%|██████████| 18881356/18881356 [00:24<00:00, 774657.23it/s]


In [51]:
seq_dict

{'ENSAPOT00000000162.1': 'SYfDYWgk&tqvTUX=^K)',
 'ENSAPOT00000002662.1': 'SYfDYWgk&tqvTUX=^K)',
 'ENSAPOT00000000435.1': '(ScM=@^XEwQtLXE=ezVukRp&esHrLTCtyzgf^GhYWNaWIrQaa&kRLEfuaXIwdSwGSIYY=EoFkG%FTIsrNnN+QQVyJQ(NwMxXEd=auyycX$HoQ)',
 'ENSAPOT00000005135.1': '^QXLtE=&z@ukrp&esHrLTctt=&jzFSSyE(SWIrQaP^k&LEWIaYiHPww^SdjYY=QzvqGRFTvsrDNSkEQLyLQ(NSLKXed=auyyca$DtQ)',
 'ENSAPOT00000010797.1': '^QXLtq=&z@ukrp&esHwLTct&=&FzFwGy=(AWIrQap&k&LeWIayi#W^dSSkyY=qzuqG%FTIsrDNnUQQLyLQ(NSLKXed=auyyca$vtQ)',
 'ENSAPOT00000013339.1': '^QXLXe=&ztuktp&esHrLTcyay&(Nf^AsW(AWIrKap^k&LeWuaXiSNdGSkyH=QzuqG%FTIsrDNNUQQLyLQ(NwLkjed=auyyc@BVtQ)',
 'ENSAPOT00000031738.1': 'NyeayFGQ&XkLXul^E}',
 'ENSAPOT00000019414.1': 'QYeayFGQ&XkLXul^E]',
 'ENSAPOT00000009432.1': 'QYeayFGQ&XkLXul^E]',
 'ENSAPOT00000009440.1': 'NyeayFGQ&XkLXul^E}',
 'ENSAPOT00000031174.1': '(qCjZmoni^yMpFMwTXn$h%PWkThIw$sJFHxApQBjWKpfVyifMwvXzaMszMazw=^FB=QEL(AIKAruQELE(eeExe%%QEEEEERcDaVD(QlLTSwp#o&PFYn(XpEERjD@DNrsvyV&nUDy&aX@dEMEIHFNGC^puNruti

In [69]:
# Get the ensemble ids
seq_keys = list(seq_dict.keys())

In [72]:
# Dictionaries
dict_df = pd.read_csv('/lustre/gleghorn/users/3018/codon_1_letter_code.csv', dtype=str)
#rna_singlecodon = dict(zip(df['Codon-RNA'], df['Single-Letter-Codon']))
#dna_singlecodon = dict(zip(dict_df['Codon-DNA'], dict_df['Single-Letter-Codon']))
#singlecodon_dna = dict(zip(df['Single-Letter-Codon'], df['Codon-DNA']))
#singlecodon_rna = dict(zip(df['Single-Letter-Codon'], df['Codon-RNA']))
#dna_amino = dict(zip(df['Codon-DNA'], df['Single-Letter-Amino']))
singlecodon_amino = dict(zip(dict_df['Single-Letter-Codon'], dict_df['Single-Letter-Amino']))
singlecodon_amino # translation between codons and amino acids

{'a': 'A',
 'A': 'A',
 '@': 'A',
 'b': 'A',
 'B': 'R',
 '#': 'R',
 '$': 'R',
 '%': 'R',
 'r': 'R',
 'R': 'R',
 'n': 'N',
 'N': 'N',
 'd': 'D',
 'D': 'D',
 'c': 'C',
 'C': 'C',
 'e': 'E',
 'E': 'E',
 'q': 'Q',
 'Q': 'Q',
 '^': 'G',
 'G': 'G',
 '&': 'G',
 'g': 'G',
 'h': 'H',
 'H': 'H',
 'i': 'I',
 'I': 'I',
 'j': 'I',
 '+': 'L',
 'M': 'L',
 'm': 'L',
 'l': 'L',
 'J': 'L',
 'L': 'L',
 'k': 'K',
 'K': 'K',
 '(': 'M',
 'f': 'F',
 'F': 'F',
 'p': 'P',
 'P': 'P',
 'o': 'P',
 'O': 'P',
 '=': 'S',
 's': 'S',
 'z': 'S',
 'Z': 'S',
 'w': 'S',
 'S': 'S',
 'X': 'T',
 'T': 'T',
 't': 'T',
 'x': 'T',
 'W': 'W',
 'y': 'Y',
 'Y': 'Y',
 'u': 'V',
 'v': 'V',
 'U': 'V',
 'V': 'V',
 ']': '*',
 '}': '*',
 ')': '*'}

 11%|█         | 998117/9440678 [02:12<08:59, 15657.59it/s]

In [81]:
# Set up dictionaries
ccds_to_uniprot = pd.read_csv('/lustre/gleghorn/users/3018/ensembl_to_uniprot.csv', dtype=str)  # load ccds ids
ccds_to_uni_dict = dict(zip(ccds_to_uniprot.iloc[:,1], ccds_to_uniprot.iloc[:,0]))  # to dictionary

In [82]:
ccds_to_uni_dict

{'ENST00000353703.9': 'P31946',
 'ENSMFAT00000073733': 'Q4R572',
 'ENSMUST00000018470.10': 'Q9CQV8',
 'ENSPPYT00000012812.3': 'A4K2U9',
 'ENSRNOT00000016981.7': 'P35213',
 'ENSBTAT00000007442.6': 'P62261',
 'ENSGALT00010070940': 'Q5ZMT0',
 'ENST00000264335.13': 'P62258',
 'ENSMUST00000067664.10': 'P62259',
 'ENSRNOT00000007100.6': 'P62260',
 'ENSOART00020017982': 'P62262',
 'ENSBTAT00000044059.4': 'P68509',
 'ENST00000248975.6': 'Q04917',
 'ENSMUST00000019109.8': 'P68510',
 'ENSRNOT00000085735.2': 'P68511',
 'ENST00000307630.5': 'P61981',
 'ENSMUST00000055808.6': 'P61982',
 'ENSRNOT00000001954.4': 'P61983',
 'ENSBTAT00000012154.5': 'Q0VC36',
 'ENST00000339276.6': 'P31947',
 'ENSMUST00000057311.4': 'O70456',
 'ENSOART00020034879': 'O77642',
 'ENSBTAT00000032851.3': 'Q3SZI4',
 'ENST00000238081.8': 'P27348',
 'ENSMUST00000103002.8': 'P68254',
 'ENSPPYT00000014710.3': 'Q5RFJ2',
 'ENSRNOT00000115392.1': 'P68255',
 'ENSBTAT00000000289.5': 'P63103',
 'ENSGALT00010028005': 'Q5ZKC9',
 'ENST0000

In [55]:
def fasta2dict(fil): # converts fasta file to dictionary of ids and sequences
    dic = {}
    cur_scaf = ''
    cur_seq = []
    for line in open(fil):
        if line.startswith(">") and cur_scaf == '':
            cur_scaf = line.split('|')[1]
        elif line.startswith(">") and cur_scaf != '':
            dic[cur_scaf] = ''.join(cur_seq)
            cur_scaf = line.split('|')[1]
            cur_seq = []
        else:
            cur_seq.append(line.rstrip())
    dic[cur_scaf] = ''.join(cur_seq)
    return dic

In [56]:
uni_sequences = fasta2dict('/lustre/gleghorn/users/3018/uniprot_sprot.fasta') # all of swiss_prot

In [97]:
translate(list(seq_dict.values())[200], singlecodon_amino) # test translation

'MGAQVQYYYERRRREALGPTVFAEEDSCHAKEECLQKILFPYLLSQLILTPELDTAVELVYLLDGEEQVTMIKKKEEKVYTAPKCLVSEKPAGLEQPKHEKCGAKINWDLSSCKGTGVWSGDLPVKLGAWPTPCQDHLTPNDQRAMLQKVKDNRKKTSAEWDS'

In [98]:
# Calculate mismatches
total, wrong = 0, 0
list_of_mismatch, list_of_mismatch_seqs, match_dist, wrong_dist = [], [], [], []
for key in tqdm(seq_keys):
    try:
        ccds_id, dna_seq = key, seq_dict[key]  # get id and sequence
        uniprot_id = ccds_to_uni_dict[ccds_id]
        prot_seq = str(uni_sequences[uniprot_id])  # get uniprot sequence of ccds id
        trans_seq = translate(dna_seq, singlecodon_amino)
        if prot_seq != trans_seq:
            wrong += 1
            list_of_mismatch.append([ccds_id, uniprot_id])
            list_of_mismatch_seqs.append([trans_seq, prot_seq])
            wrong_dist.append(len(mismatch_count(trans_seq, prot_seq)))
        else:
            wrong_dist.append(0)
        total += 1
        match_dist.append(0)
    except:
        continue

print(total, wrong)




  0%|          | 0/9440678 [00:00<?, ?it/s]


  1%|          | 62584/9440678 [00:00<00:14, 625833.14it/s]


  1%|▏         | 125719/9440678 [00:00<00:14, 627414.44it/s]


  2%|▏         | 189152/9440678 [00:00<00:14, 629472.51it/s]


  3%|▎         | 250068/9440678 [00:00<00:14, 622854.79it/s]


  3%|▎         | 314515/9440678 [00:00<00:14, 629184.86it/s]


  4%|▍         | 381853/9440678 [00:00<00:14, 641819.15it/s]


  5%|▍         | 461506/9440678 [00:00<00:13, 681529.60it/s]


  6%|▌         | 529792/9440678 [00:00<00:13, 681927.89it/s]


  6%|▋         | 601152/9440678 [00:00<00:12, 691127.18it/s]


  7%|▋         | 685107/9440678 [00:01<00:11, 729834.85it/s]


  8%|▊         | 765142/9440678 [00:01<00:11, 749648.49it/s]


  9%|▉         | 839600/9440678 [00:01<00:16, 524602.59it/s]


 10%|▉         | 913210/9440678 [00:01<00:14, 574086.02it/s]


 11%|█         | 993950/9440678 [00:01<00:13, 628493.78it/s]


 11%|█▏        | 1063691/9440678 [00:01<00:12, 645011.97it/s]


 12%|█

39798 39798


In [101]:
# Write results
with open('./results_cds_mismatches.txt', 'w') as f:
    for i in range(len(list_of_mismatch_seqs)):
        ccds = list_of_mismatch_seqs[i][0]
        uni = list_of_mismatch_seqs[i][1]
        mismatches = mismatch_count(ccds, uni)
        f.write(list_of_mismatch[i][0] + '\t' + list_of_mismatch[i][1] + '\n' + 'CCDS:    ' + ccds + '\n' + 'UniProt: ' + uni + '\n')
        f.write(str(len(mismatches)) + '\n')
        f.write(str(mismatches) + '\n')